In [ ]:
import pandas as pd
import numpy as np
from pykrige.ok import OrdinaryKriging
import matplotlib.pyplot as plt
import os

# --- Konfigurasi ---
# Pastikan file ini ada di direktori yang sama dengan skrip
KEJADIAN_FILE = 'data/kejadian.csv' 
OUTPUT_PLOT_FILE = 'semivariogram_eksperimental.png'
# --------------------

print(f"--- Skrip Plot Semivariogram Eksperimental Dimulai ---")

# Validasi keberadaan file
if not os.path.exists(KEJADIAN_FILE):
    print(f"❌ ERROR: File {KEJADIAN_FILE} tidak ditemukan.")
    print("Pastikan file 'kejadian.csv' ada di direktori yang sama dengan skrip ini.")
else:
    print(f"✓ File {KEJADIAN_FILE} ditemukan.")
    
    try:
        # Langkah 1: Memuat data
        print("\nLangkah 1: Memuat data dari CSV...")
        df_crime = pd.read_csv(KEJADIAN_FILE)

        # Langkah 2: Mempersiapkan data untuk Kriging
        print("\nLangkah 2: Mempersiapkan data (lons, lats, values)...")
        # Mengambil data dari DataFrame
        lons = df_crime['longitude'].values
        lats = df_crime['latitude'].values
        values = df_crime['jumlah_kejadian'].values
        print(f"✓ {len(values)} titik data siap untuk diproses.")

        # Langkah 3: Menghitung Semivariogram Eksperimental
        print("\nLangkah 3: Menghitung semivariogram...")
        # Inisialisasi OrdinaryKriging untuk menghitung
        # semivariogram eksperimental secara internal.
        # Kita set enable_plotting=False agar plot tidak otomatis muncul
        OK = OrdinaryKriging(
            lons, lats, values,
            variogram_model="spherical", # Sesuai skripsi Anda
            verbose=False,       
            enable_plotting=False 
        )

        # Langkah 4: Mengambil data plot (lags dan semivariance)
        print("Langkah 4: Mengambil data hasil perhitungan (lags dan semivariance)...")
        # Data ini (lags dan semivariance) adalah yang Anda butuhkan untuk plot titik
        lags = OK.lags
        semivariance = OK.semivariance

        # Langkah 5: Membuat dan menyimpan plot
        print(f"\nLangkah 5: Membuat plot dan menyimpannya sebagai '{OUTPUT_PLOT_FILE}'...")
        
        # Membuat canvas plot
        plt.figure(figsize=(10, 6))
        
        # 'bo' berarti 'blue' (b) markers 'o' (o) -> ini adalah 'plot titik'
        plt.plot(lags, semivariance, 'bo', markersize=8, alpha=0.7, label='Data Eksperimental')
        
        # Memberi Judul dan Label
        plt.title('Plot Semivariogram Eksperimental', fontsize=16)
        plt.xlabel('Jarak Antar Titik (h)', fontsize=12)
        plt.ylabel('Semivarians $\gamma(h)$', fontsize=12) # Menggunakan LaTeX
        
        # Menambahkan grid untuk kemudahan pembacaan
        plt.grid(True, linestyle='--', alpha=0.6)
        plt.legend()
        
        # Menyimpan file plot
        plt.savefig(OUTPUT_PLOT_FILE)
        
        print(f"\n--- Selesai ---")
        print(f"✓ Plot telah berhasil dibuat dan disimpan sebagai: {OUTPUT_PLOT_FILE}")

    except Exception as e:
        print(f"❌ Terjadi error: {e}")